In [ ]:
%matplotlib inline

Guardar y cargar el modelo
 ============================

 En esta sección, veremos cómo conservar el estado del modelo guardando, cargando y ejecutando predicciones del modelo.


In [4]:
import torch
import torchvision.models as models
import torch.nn as nn

Guardar y cargar pesos de modelo
 --------------------------------
 Los modelos PyTorch almacenan los parámetros aprendidos en un
 diccionario de estado, llamado ``state_dict``.  Estos pueden persistir a través de ``torch.save``
 método:



In [ ]:
model = models.vgg16(pretrained=True)
torch.save(model.state_dict(), 'model_weights.pth')

Para cargar los pesos del modelo, primero debe crear una instancia del mismo modelo y luego cargar los parámetros.
 utilizando el método ``load_state_dict()``.



In [ ]:
model = models.vgg16() # no especificamos pretrained=True, es decir, no cargamos pesos predeterminados
model.load_state_dict(torch.load('model_weights.pth'))
model.eval()

<div class="alert alert-info"><h4>Note</h4><p>be sure to call ``model.eval()`` method before inferencing to set the dropout and batch normalization layers to evaluation mode. Failing to do this will yield inconsistent inference results.</p></div>



Guardar y cargar modelos con formas
 -------------------------------------
 Al cargar los pesos del modelo, primero necesitábamos crear una instancia de la clase del modelo, porque la clase
 define la estructura de una red.  Podríamos querer guardar la estructura de esta clase junto con
 el modelo, en cuyo caso podemos pasar ``model`` (y no ``model.state_dict()``) a la función de guardado:



In [ ]:
torch.save(model, 'model.pth')

Entonces podemos cargar el modelo así:


In [ ]:
model = torch.load('model.pth')

<div class="alert alert-info"><h4>Nota</h4><p>Este enfoque utiliza Python`pickle <https://docs.python.org/3/library/pickle.html>`_ módulo al serializar el modelo, por lo tanto, se basa en la definición de clase real para estar disponible al cargar el modelo.</p></div>



Guardar y cargar Puntos del Entrenamiento
 -------------------------------------
 
 Imaginemos que tenmos el siguiente modelo:

In [6]:
class Model(nn.Module):
    def __init__(self, n_input_features):
        super(Model, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)

    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

model = Model(n_input_features=6)

learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

**Guardamos los parametros bajo los cuales queremos hacer nuestro punto de control**

In [7]:
checkpoint = {
"epoch": 90,
"model_state": model.state_dict(),
"optim_state": optimizer.state_dict()
}
print(optimizer.state_dict())

{'state': {}, 'param_groups': [{'lr': 0.01, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'maximize': False, 'params': [0, 1]}]}


**Salvamos nuestro modelo**

In [8]:
FILE = "checkpoint.pth"
#torch.save(Parametros de guardado, Nombre del archivo)
torch.save(checkpoint, FILE)

**Supongamos que nuestro modelo queda asi**

In [9]:
model = Model(n_input_features=6)
optimizer = optimizer = torch.optim.SGD(model.parameters(), lr=0)

**Asi cargariamos nuestro modelo**

In [10]:
#Cargar modelo
checkpoint = torch.load(FILE)

#Cargar el estado del modelo
model.load_state_dict(checkpoint['model_state'])

#Cargar el optimizador del modelo
optimizer.load_state_dict(checkpoint['optim_state'])

#Cargar la epoca del modelo
epoch = checkpoint['epoch']

#Entrenamos nuestro modelo
# model.train()
# - or -
model.eval()

Model(
  (linear): Linear(in_features=6, out_features=1, bias=True)
)